In [1]:
# IMPORTS
import numpy as np
import cv2
import time

In [13]:
# Carga de la YOLO v3
#Nombres de las clases
with open("/content/drive/MyDrive/movil-final/imagenes/"+ "class.names") as f:
    labels = [line.strip() for line in f]


network = cv2.dnn.readNetFromDarknet("/content/drive/MyDrive/movil-final/"+ "cell-yolov3.cfg",
                                     "/content/drive/MyDrive/movil-final/backup/" + "cell-yolov3_best.weights")

layers_names_all = network.getLayerNames()

layers_names_output = \
    [layers_names_all[i - 1] for i in network.getUnconnectedOutLayers()]

probability_minimum = 0.3
threshold = 0.3

colours = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')


In [14]:
def movil_detection(nombre_video):
  # CARGA DEL VIDEO Y LOOP DE LOS FRAMES
  f=0
  t=0
  writer = None
  h, w = None, None
  video = cv2.VideoCapture("/content/drive/MyDrive/movil-final/videos/entrenamiento/"+ nombre_video)

  while True:
    ret, frame=video.read()

    if not ret:
      break

    if w is None or h is None:
      h,w = frame.shape[:2]

    blob = cv2.dnn.blobFromImage(frame, 1/ 255.0, (416,416), swapRB=True, crop=False)

    #APLICACIÓN DEL NETWORK A LOS LOOPS
    network.setInput(blob)
    start=time.time()
    output_from_network = network.forward(layers_names_output)
    end=time.time()

    f+=1
    t+= end-start


    bounding_boxes = []
    confidences = []
    class_numbers = []

    for result in output_from_network:
        for detected_objects in result:
            scores = detected_objects[5:]
            class_current = np.argmax(scores)
            
            confidence_current = scores[class_current]

            if confidence_current > probability_minimum:
                box_current = detected_objects[0:4] * np.array([w, h, w, h])

                x_center, y_center, box_width, box_height = box_current
                x_min = int(x_center - (box_width / 2))
                y_min = int(y_center - (box_height / 2))

                bounding_boxes.append([x_min, y_min,
                                       int(box_width), int(box_height)])
                confidences.append(float(confidence_current))
                class_numbers.append(class_current)


# FILTRADO DE LA MEJOR CAJA Y DIBUJADO DE ESTA 
    results = cv2.dnn.NMSBoxes(bounding_boxes, confidences,
                               probability_minimum, threshold)    
    
    if len(results) > 0:
        
        for i in results.flatten():
            x_min, y_min = bounding_boxes[i][0], bounding_boxes[i][1]
            box_width, box_height = bounding_boxes[i][2], bounding_boxes[i][3]

            colour_box_current = colours[class_numbers[i]].tolist()

            if   labels[int(class_numbers[i])]=="cell phone":
              cv2.rectangle(frame, (x_min, y_min),
                          (x_min + box_width, y_min + box_height),
                          colour_box_current, 2)              
              text_box_current = '{}: {:.4f}'.format(labels[int(class_numbers[i])],
                                                    confidences[i])

              cv2.putText(frame, text_box_current, (x_min, y_min - 5),
                          cv2.FONT_HERSHEY_SIMPLEX, 0.5, colour_box_current, 2) 

    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')

        writer = cv2.VideoWriter("/content/drive/MyDrive/movil-final/resultados/" + nombre_video, fourcc, 30,
                                 (frame.shape[1], frame.shape[0]), True)
    writer.write(frame)   
  print()
  print("Nombre_archivo:", nombre_video)
  print('Numero de imágenes', f)
  print('Tiempo en Minutos: {:.5f}'.format(t/60))
  print('FPS:', round((f / t), 1))


In [15]:
lista_videos4=["movil1.mp4","movil2.mp4","movil3.mp4","movil4.mp4","movil5.mp4","movil6.mp4"]
for x in lista_videos4:
  movil_detection(x)


Nombre_archivo: movil1.mp4
Numero de imágenes 573
Tiempo en Minutos: 11.17839
FPS: 0.9

Nombre_archivo: movil2.mp4
Numero de imágenes 826
Tiempo en Minutos: 16.11816
FPS: 0.9

Nombre_archivo: movil3.mp4
Numero de imágenes 672
Tiempo en Minutos: 13.11464
FPS: 0.9

Nombre_archivo: movil4.mp4
Numero de imágenes 269
Tiempo en Minutos: 5.23140
FPS: 0.9

Nombre_archivo: movil5.mp4
Numero de imágenes 577
Tiempo en Minutos: 11.27834
FPS: 0.9

Nombre_archivo: movil6.mp4
Numero de imágenes 920
Tiempo en Minutos: 17.92388
FPS: 0.9
